In [4]:
#importing packages etc.
import numpy as numpy
import math as math
import sys

#variable declaration

m=4
ndof=2

Lx=1.e0
Ly=1.e0

nnx=21
nny=21

np=nnx*nny

nelx=nnx-1
nely=nny-1

nel=nelx*nely

penalty=1.e7

viscosity=1.e0
density=1.e0

Nfem=np*ndof

eps=1.e-10

gx=0
gy=1

Kmat = numpy.zeros(shape=(3,3))
Cmat = numpy.zeros(shape=(3,3))

Kmat[0,0]=1.e0 ; Kmat[0,1]=1.e0 ; Kmat[0,2]=0.e0  
Kmat[1,0]=1.e0 ; Kmat[1,1]=1.e0 ; Kmat[1,2]=0.e0  
Kmat[2,0]=0.e0 ; Kmat[2,1]=0.e0 ; Kmat[2,2]=0.e0  

Cmat[0,0]=2.e0 ; Cmat[0,1]=0.e0 ; Cmat[0,2]=0.e0  
Cmat[1,0]=0.e0 ; Cmat[1,1]=2.e0 ; Cmat[1,2]=0.e0  
Cmat[2,0]=0.e0 ; Cmat[2,1]=0.e0 ; Cmat[2,2]=1.e0  

# print ("Kmat=")
# print (Kmat)

# print ("Cmat=")
# print (Cmat)

In [12]:
#declaring arrays
x      = numpy.zeros((np),dtype=float)
y      = numpy.copy(x)
u      = numpy.copy(x)
v      = numpy.copy(x)
A      = numpy.zeros((Nfem,Nfem),dtype=float)
B      = numpy.zeros((Nfem),dtype=float)
icon   = numpy.zeros((m,nel),dtype=int)
bc_fix = numpy.zeros((Nfem),dtype=bool)
bc_val = numpy.zeros((Nfem),dtype=float)
Ael    = numpy.zeros((m*ndof,m*ndof),dtype=float)
Bel    = numpy.zeros((m*ndof),dtype=float)
N      = numpy.zeros((m),dtype=float)
dNdx   = numpy.copy(N)
dNdy   = numpy.copy(N)
dNdr   = numpy.copy(N)
dNds   = numpy.copy(N)
jcb    = numpy.zeros((2,2),dtype=float)
jcob   = numpy.copy(jcb)
jcbi   = numpy.copy(jcb)
Bmat   = numpy.zeros((3,ndof*m),dtype=float)
Cmat   = numpy.zeros((3,3),dtype=float)

In [13]:
#grid point setup
counter=0
for j in range (0,nely+1):
    for i in range (0,nelx+1):
        x[counter]=i*Lx/nelx
        y[counter]=j*Ly/nely
        counter += 1

# for i in range (0,np):
#     print (x[i],y[i],i)

In [14]:
#connectivity
counter=0
for j in range (0,nely):
    for i in range (0,nelx):
        icon[0][counter]=i+   j*(nelx+1)
        icon[1][counter]=i+1+ j*(nelx+1)
        icon[2][counter]=i+1+(j+1)*(nelx+1)
        icon[3][counter]=i  +(j+1)*(nelx+1)
        counter += 1

# for iel in range (0,nel):
#         print ("iel=",iel)
#         print ("node 1",icon[0][iel],"at pos.",x[icon[0][iel-1]], y[icon[0][iel-1]])
#         print ("node 2",icon[1][iel],"at pos.",x[icon[1][iel-1]], y[icon[1][iel-1]])
#         print ("node 3",icon[2][iel],"at pos.",x[icon[2][iel-1]], y[icon[2][iel-1]])
#         print ("node 4",icon[3][iel],"at pos.",x[icon[3][iel-1]], y[icon[3][iel-1]])

In [15]:
#define boundary conditions

for i in range (0,np):
    if x[i] < eps:
        bc_fix[i*ndof] = True   ; bc_val[i*ndof]=0.e0
        bc_fix[i*ndof+1]=True ; bc_val[i*ndof+1]=0.e0
    if x[i] > (Lx-eps):
        bc_fix[i*ndof] = True   ; bc_val[i*ndof]=0.e0
        bc_fix[i*ndof+1]=True ; bc_val[i*ndof+1]=0.e0
    if y[i] < eps:
        bc_fix[i*ndof] = True   ; bc_val[i*ndof]=0.e0
        bc_fix[i*ndof+1]=True ; bc_val[i*ndof+1]=0.e0
    if y[i] > (Ly-eps):
        bc_fix[i*ndof] = True   ; bc_val[i*ndof]=0.e0
        bc_fix[i*ndof+1]=True ; bc_val[i*ndof+1]=0.e0

# for i in range (0,np):
#     if bc_fix[i*ndof] == True:  
#         print (counter,x[i],y[i],u[i])
#     if bc_fix[i*ndof+1] == True:
#         print (counter,x[i],y[i],v[i])

In [34]:
#build FE matrix

for iel in range (0,nel):
    for iq in range (-1,3,2):
        for jq in range (-1,3,2):
            rq=iq/math.sqrt(3.e0)
            sq=jq/math.sqrt(3.e0)
            wq=1.e0*1.e0

            N[0]=0.25e0*(1.e0-rq)*(1.e0-sq)
            N[1]=0.25e0*(1.e0+rq)*(1.e0-sq)
            N[2]=0.25e0*(1.e0+rq)*(1.e0+sq)
            N[3]=0.25e0*(1.e0-rq)*(1.e0+sq)

            dNdr[0]= - 0.25e0*(1.e0-sq)   ;   dNds[0]= - 0.25e0*(1.e0-rq)
            dNdr[1]= + 0.25e0*(1.e0-sq)   ;   dNds[1]= - 0.25e0*(1.e0+rq)
            dNdr[2]= + 0.25e0*(1.e0+sq)   ;   dNds[2]= + 0.25e0*(1.e0+rq)
            dNdr[3]= - 0.25e0*(1.e0+sq)   ;   dNds[3]= + 0.25e0*(1.e0-rq)
            
            
            for k in range (0,m):
                jcb[0,0]=jcb[0,0]+dNdr[k]*x[icon[k,iel]]
                jcb[0,1]=jcb[0,1]+dNdr[k]*y[icon[k,iel]]
                jcb[1,0]=jcb[1,0]+dNds[k]*x[icon[k,iel]]
                jcb[1,1]=jcb[1,1]+dNds[k]*y[icon[k,iel]]
      
            jcob=jcb[0,0]*jcb[1,1]-jcb[1,0]*jcb[0,1]

            jcbi[0,0]=    jcb[1,1] /jcob
            jcbi[0,1]=  - jcb[0,1] /jcob
            jcbi[1,0]=  - jcb[1,0] /jcob
            jcbi[1,1]=    jcb[0,0] /jcob
            
            xq=0.e0
            yq=0.e0
            uq=0.e0
            vq=0.e0
            exxq=0.e0
            eyyq=0.e0
            exyq=0.e0
            for k in range(0,m):
                xq=xq+N[k]*x[icon[k,iel]]
                yq=yq+N[k]*y[icon[k,iel]]
                uq=uq+N[k]*u[icon[k,iel]]
                vq=vq+N[k]*v[icon[k,iel]]
                dNdx[k]=jcbi[0,0]*dNdr[k]+jcbi[0,1]*dNds[k]
                dNdy[k]=jcbi[1,0]*dNdr[k]+jcbi[1,1]*dNds[k]
                exxq=exxq+ dNdx[k]*u[icon[k,iel]]
                eyyq=eyyq+ dNdy[k]*v[icon[k,iel]]
                exyq=exyq+ dNdx[k]*v[icon[k,iel]] *0.5e0 + dNdy[k]*u[icon[k,iel]] *0.5e0
                
            for i in range(0,m):
                i1=2*i
                i2=2*i+1
                Bmat[0,i1]=dNdx[i] ; Bmat[0,i2]=0.e0
                Bmat[1,i1]=0.e0    ; Bmat[1,i2]=dNdy[i]
                Bmat[2,i1]=dNdy[i] ; Bmat[2,i2]=dNdx[i]
                
            #Ael=Ael + Bmat.transpose()*(viscosity*Cmat)*Bmat*wq*jcob
            Ael=numpy.matmul(Bmat.transpose(),viscosity*numpy.matmul(Cmat,Bmat))*wq*jcob
            
            
            for i in range(0,m):
                i1=2*i
                i2=2*i+1
                Bel[i1]=Bel[i1]+N[i]*jcob*wq*density*gx
                Bel[i2]=Bel[i2]+N[i]*jcob*wq*density*gy
                #Bel[i1]=Bel[i1]+N[i]*jcob*wq*b1[xq,yq]
                #Bel[i2]=Bel[i2]+N[i]*jcob*wq*b2[xq,yq]